\begin{equation*}
\newcommand{\E}{\mathbb{E}}
\newcommand{\Nor}{\mathcal{N}}
\end{equation*}




# Due Dates

* Textbook: Thursday, November 1, in class
* Coding: Thursday, November 1, at 11:59 PM

# Textbook Problems

These problems are also coding-based. It may be helpful to do the first two coding exercises, which are related. 

* 3.39
* 3.41
* 3.43 (Use an Euler approximation to compute the time updates in discrete-time. The continuous-discrete unscented Kalman filter is outside of the scope of the class.)



In [1]:
# This is code to load the assignment.
# You'll need to run this code do or restart the assignment.
from loadAssignment import loadAssignment
Assignment, Questions, Submit, Data = loadAssignment(6)

# These are modules that we need
# once you run this code, you don't need to load them again
import autograd.numpy as np
import autograd as ag
import scipy.linalg as la
import scipy.integrate as itg
import matplotlib.pyplot as plt
%matplotlib inline




# Question 0





The first few problems in this assignment will be a warm-ups for the texbook problems. In particular, they will focus on coding continuous-time simulations of the Van der Pol System from example 3.5. That example focuses on the case of estimating the states of a nonlinear dynamic system when the parameters are incorrectly specified.

Assume that the initial condition is $x(0)=\left[\begin{matrix}1.0\\0.0\end{matrix}\right]$. 
Use [`itg.solve_ivp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) to simulate the true Van der Pol system for 1000 time points spaced according to $\Delta t = 0.01$. Make a plot with $x_1(t)$ on the $x$-axis and $x_2(t)$ on the $y$-axis. 



In [ ]:
# Make your plot here

Questions[0].checkAnswer()


# Question 1

Assume that no measurements are taken. Using just the propagation equation from Table 3.9, the mean $\hat x(t)$ and $ P(t)$ can be simulated. Now use `itg.solve_ivp` to simulate $\hat x(t)$ and $P(t)$. As in example 3.5, for this simulation, assume that the mis-specified values are used. Also, as in the example, assume that $q =0.2$ and $P(0) = 1000 I$. Assume that $\hat x(0) = \left[\begin{matrix}1.0\\0.0\end{matrix}\right]	$

Note that in order to use `itg.solve_ivp`, you will need to flatten $\hat x(t)$ and $P(t)$ into a single vector.

Make two plots. The first plot should show $x_1(t)$ and $\hat x(t)$ on the $x$-axis and $x_2(t)$ and $\hat x_2(t)$ on the $y$-axis. The second plot should have time on the $x$ axis (starting from $0$) and the 3 unique entries of $P(t)$ on the $y$ axis.



In [ ]:
fig_mean = plt.figure()
# Make your plots of x and x_hat here

fig_cov = plt.figure()
# Make your covariance plots here


Questions[1].checkAnswer(fig_mean,fig_cov)


# Question 2

For the rest of the assignment, we will see how to apply the unscented Kalman filter to the localization problem from the previous homework.

Recall the vehicle model given in discrete-time by:
\begin{equation*}
\begin{bmatrix}
p_x[k+1] \\
p_y[k+1] \\
\theta[k+1]
\end{bmatrix}
=
\begin{bmatrix}
p_x[k] \\
p_y[k] \\
\theta[k]
\end{bmatrix} + dt 
\begin{bmatrix}
u_v[k] \cos(\theta[k]) \\ 
u_v[k] \sin(\theta[k]) \\
u_\omega[k]
\end{bmatrix}
+w[k].
\end{equation*}




Here $dt$ is the time-step and $w[k]$ is the process noise. We will assume that $dt=0.1$ and the process noise is independent Gaussian noise distributed as $w[k] \sim\Nor(0,0.0001I)$.

We stack the state and inputs as:
\begin{equation*}
x[k] = \begin{bmatrix}
p_x[k] \\
p_y[k] \\
\theta[k]
\end{bmatrix}
\quad 
u[k] = \begin{bmatrix}
u_v[k] \\
u_\omega[k]
\end{bmatrix}
\end{equation*}

This question will focus on the time-update.

Throughout this assignment, we will be implementing Algorithm 5.15 from:

https://users.aalto.fi/~ssarkka/pub/cup_book_online_20131111.pdf

(This algorithm is somewhat different from the description of the UKF from the course text. 

Assume that we have computed our posterior approximations:
\begin{equation*}
\hat x_k^+ = \E[x_k | y_{0:k},u_{0:k-1}], \quad P_k^+ = \E[(x_k-\hat x_k^+)(x_k-\hat x_k^+)^\top]
\end{equation*}

Given the input $u_k$ calculate the UKF approximation to the mean can covariance:
\begin{equation*}
\hat x_{k+1}^- = \E[x_{k+1} | y_{0:k},u_{0:k}], \quad 
P_{k+1}^- = \E[(x_{k+1}-\hat x_{k+1}^-)(x_{k+1}-\hat x_{k+1}^-)^\top]
\end{equation*}

Specifically, write a function of the form:

```
x_pre,P_pre = carTimeUpdateUKF(x_post,P_post,u)
```

Use the lower triangular Cholesky decomposition from [`sp.cholesky`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.cholesky.html) to compute the sigma points. (Note, matrix square roots are not unique. So, for the checker to work we all need to use the same code.)

For all of the unscented transforms in this homework, use the parameters:
\begin{equation*}
\alpha = 1, \quad \beta = 2, \quad \kappa = 3-n,
\end{equation*}
where $n$ is the dimension of the variable which you are transforming.






In [ ]:
# Define your function here

Questions[2].checkAnswer(carTimeUpdateUKF)


# Question 3

Recall the distance measurement model from last week ago:

\begin{equation*}
(y_k)_i = \|p_k-c_i\| + 0.2 (1+\|p_k-c_i\|) (v_k)_i
\end{equation*}

Here $p_k = \begin{bmatrix} p_x[k] \\ p_y[k] \end{bmatrix}$, $c_i$ corresponds to the location of a sensor, and $(v_k)_i$ are independent Gaussian noises with mean $0$ and covariance $1$. The term $(y_k)_i$ denotes that entry $i$ of the measurement at time $k$. Similarly, $(v_k)_i$ is the entry $i$ of the measurement noise at time $k$. 
In this problem, we will have $10$ sensors at locations given below:



In [ ]:
SensorLocs = Data.SensorLocs


Assume now that the prior approximations have been computed:

\begin{equation*}
\hat x_k^- = \E[x_k | y_{0:k-1},u_{0:k-1}], \quad P_k^- = \E[(x_k - \hat x_k^-)(x_k - \hat x_k^-)^\top]
\end{equation*}

Given the measurement, $y_k$, compute the UKF posterior approximations:

\begin{equation*}
\hat x_k^+ = \E[x_k | y_{0:k},u_{0:k-1}], \quad P_k^+ = \E[(x_k-\hat x_k^+)(x_k - \hat x_k^+)^\top].
\end{equation*}

Specifically, write a function of the form:

```
x_post,P_post = carMeasUpdateUKF(x_pre,P_pre,y)
```



In [ ]:
# Define your function here


Questions[3].checkAnswer(carMeasUpdateUKF)




# Question 4

Now we will use the functions above to compute the UKF for the vehicle model. So, we will assume that the vehicle state is no longer directly measured. Assume that the initial state is distributed as $\Nor(0,10\cdot I)$. Furthermore, assume that the inputs and measurements are those given below:



In [ ]:
U = Data.U
Y = Data.Y


In particular, make a $490\times 3$ array for the values of $\hat x_k^+$ and an $490 \times 3\times 3$ array for the values of $P_k^+$. 



In [ ]:
# Calculate the estimates here
# Call them Mu and P_arr

Questions[4].checkAnswer(Mu,P_arr)


Below is a plot of the performance of the UKF



In [ ]:
X = Data.X

fig,ax = plt.subplots(3,1,sharex=True)
Time = 0.1 * np.arange(len(X))
ax[0].plot(Time,X[:,0])
ax[0].plot(Time,Mu[:,0])
ax[1].plot(Time,X[:,1])
ax[1].plot(Time,Mu[:,1])

shiftAngle = lambda theta : ((theta + np.pi) % (2*np.pi)) -np.pi

ax[2].plot(Time,shiftAngle(X[:,2]))
ax[2].plot(Time,shiftAngle(Mu[:,2]))


# Final Score

You can run this code to see all of your scores.




In [ ]:
Assignment.showResults()




# Submission

Save your work and run this cell to submit. It will only work if you have the internet.



In [ ]:
Submit()